In [1]:
import os, glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch

from ultralytics import YOLO
import easyocr
import utils

<h3 style="color:orange">Hyperparameters</h3>

In [2]:
DEVICE="cuda" if torch.cuda.is_available() else "cpu"

reader=easyocr.Reader(['en'],gpu=torch.cuda.is_available())

print(f"{DEVICE=}")

vid_dir=["./dataset/video/demo.mp4"]

vid=cv2.VideoCapture(vid_dir[0])

# video metadata
frame_count=int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
W=int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
H=int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
FPS=int(vid.get(cv2.CAP_PROP_FPS))

print(f"frame count: {frame_count}\nframe width: {W}\nframe hight: {H}\nFPS: {FPS}")

DEVICE='cuda'
frame count: 631
frame width: 1114
frame hight: 720
FPS: 30


<h3 style="color:orange">Load Model</h3>

In [3]:
model_weight=[path for path in glob.glob("./Model/weights/*.pt")]
print(model_weight)
model=YOLO(model_weight[1]).to(DEVICE)

matter_tracker=utils.sort.Sort()   #use for tracting object in continues frames

print(model)

['./Model/weights\\yolov8n.pt', './Model/weights\\YOLOv8_LPD_largeset.pt', './Model/weights\\YOLOv8_LPD_roboflow.pt']
YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1,

<h3 style="color:orange">model infernce</h3>

In [ ]:
results={}
LIMIT=50

ret,frame_num=True,-1
while(ret):
    frame_num+=1
    ret,frame=vid.read()
    print(f"{ret}, Frame no. {frame_num}")
    if(ret):

        if(frame_num>LIMIT): break     #debug
        
        results[frame_num]={}

        # detect numberplate
        detections=model(frame)[0]
        detection_list=[]
        for detection in detections.boxes.data.tolist():
            x1,y1,x2,y2,score,class_id=detection
            if(score>=0.5):
                # print([int(x1),int(y1),int(x2),int(y2),score])
                detection_list.append([int(x1),int(y1),int(x2),int(y2),score])
        print("\n")
    

        ## track vehicle 
        # track_id=matter_tracker.update(np.asarray(detection_list))
        # print(track_id)

        # crop Lincence plate
        corp_license_plate=frame[int(y1):int(y2),int(x1):int(x2),:]
        # print(corp_license_plate.shape)
        ## cv2.imwrite(f"./temp/{frame_num}.png",corp_license_plate)    ## DEBUG

        ## correct one
        # ans=utils.utils.read_license_plate(corp_license_plate)
        
        # wrting into video frame
        detect=reader.readtext(corp_license_plate)
        for x in detect:
            if(len(x)!=0):
                print(x)
                temp_frame=frame.copy()
                cv2.rectangle(temp_frame,(int(x1),int(y1)),(int(x2),(int(y2))),(0,255,255),2)
                cv2.imwrite(f"./temp/ans/{frame_num}.png",temp_frame)

        continue
        

True, Frame no. 0

0: 416x640 1 License Plates, 45.3ms
Speed: 6.7ms preprocess, 45.3ms inference, 3.2ms postprocess per image at shape (1, 3, 416, 640)


([[np.int32(3), np.int32(3)], [np.int32(157), np.int32(3)], [np.int32(157), np.int32(41)], [np.int32(3), np.int32(41)]], 'L656-XH', np.float64(0.617456362196487))
True, Frame no. 1

0: 416x640 1 License Plates, 41.6ms
Speed: 4.9ms preprocess, 41.6ms inference, 2.3ms postprocess per image at shape (1, 3, 416, 640)


([[np.int32(17), np.int32(3)], [np.int32(167), np.int32(3)], [np.int32(167), np.int32(39)], [np.int32(17), np.int32(39)]], 'L656.XH', np.float64(0.5584951972002777))
True, Frame no. 2

0: 416x640 2 License Platess, 22.5ms
Speed: 3.8ms preprocess, 22.5ms inference, 2.4ms postprocess per image at shape (1, 3, 416, 640)


([[np.int32(13), np.int32(3)], [np.int32(159), np.int32(3)], [np.int32(159), 37], [np.int32(13), 37]], 'L-656-H', np.float64(0.6015533189575825))
True, Frame no. 3

0: 416x640 2 License Platess, 21.9ms
Speed: